In [ ]:
import pandas as pd
from sklearn.metrics import r2_score
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import scipy.optimize
import scipy.special
import scipy.linalg
from tqdm.notebook import tqdm

# load CR9114
df = pd.read_csv("20210427_HA_unadj_fil_merg.csv", dtype={'variant':'str'}
        )

## Note that the CR9114 dataset is huge, so this code needs quite a bit of memory to run correctly

In [ ]:
def hadamard(v, L):
    """ Hadamard matrix multiplication, but without the same memory impact """
    if L == 0:
        return v
    a1 = 1/2*hadamard(v[:2**(L-1)], L-1)
    a2 = 1/2*hadamard(v[2**(L-1):], L-1)
    return np.concatenate((a1+a2, -a1+a2), axis=0) 


## define the different epistatic inference methods

def RB_LS_inference(measured_phenotypes, genotypes, degree, L, test_frac=0):
    """ Reference-based, least-square method (Original) 
        degree: maximum order of the model
        L: number of sites
        test_frac: proportion of sites in the test subset (in a test/train formalism)
    """
    poly = PolynomialFeatures(degree, interaction_only=True)
    ## select a random 1/5 subset of the coefficients for testing
    int_genotypes = [int(a, 2) for a in genotypes]
    idx_test_geno = np.random.choice(range(len(genotypes)),  int(len(measured_phenotypes)*test_frac), replace=False)
    test_geno = [int_genotypes[ii] for ii in idx_test_geno]
    idx_train_geno = [a for a in range(len(genotypes)) if a not in idx_test_geno]
    train_geno = [int_genotypes[ii] for ii in idx_train_geno]
    
    geno = poly.fit_transform([[int(a) for a in f'{b:0{L}b}'] for b in train_geno])
    result_opt = scipy.linalg.lstsq(
        geno, measured_phenotypes[idx_train_geno])
    res = result_opt[0]
    
    ## measure the r2 on the test and train set
    train_pred = geno @ res
    # Calculating R-squared
    r2_train = r2_score(measured_phenotypes[idx_train_geno], train_pred)

    r2_test = 0
    if test_frac != 0:
        poly = PolynomialFeatures(degree, interaction_only=True)
        test_geno = poly.fit_transform([[int(a) for a in f'{b:0{L}b}'] for b in test_geno])
        test_pred = test_geno @ res
        r2_test = r2_score(measured_phenotypes[idx_test_geno], test_pred)
      
    return res, r2_train, r2_test

def RBA_inference(measured_phenotypes, genotypes, degree, L):
    """
    RBA inference, up to order `degree`, with L sites.
    """
    poly = PolynomialFeatures(degree, interaction_only=True)
    lowdegree_values = [int(b, 2) for b in genotypes if b.count('1') <= degree]
    idx_lowdegree_values = [ii for ii, b in enumerate(genotypes) if b.count('1') <= degree]
    geno = poly.fit_transform([[int(a) for a in f'{b:0{L}b}'] for b in lowdegree_values])
    result_opt = scipy.linalg.lstsq(
        geno, measured_phenotypes[idx_lowdegree_values])
    res = result_opt[0]
    ## measure the r2
    geno = poly.fit_transform([[int(a) for a in f'{b:0{L}b}'] for b in range(2**L)])
    pred = geno @ res

    # Calculating R-squared
    r2 = r2_score(measured_phenotypes, pred)

    return res, r2

def RFA_inference(measured_phenotypes, genotypes, degree, L, test_frac=0):
    """ Reference-free, least-square method (RFA) 
        degree: maximum order of the model
        L: number of sites
        test_frac: proportion of sites in the test subset (in a test/train formalism)
    """
    poly = PolynomialFeatures(degree, interaction_only=True)
    ## select a random 1/5 subset of the coefficients for testing
    int_genotypes = [int(a, 2) for a in genotypes]
    idx_test_geno = np.random.choice(range(len(genotypes)),  int(len(measured_phenotypes)*test_frac), replace=False)
    test_geno = [int_genotypes[ii] for ii in idx_test_geno]
    idx_train_geno = [a for a in range(len(genotypes)) if a not in idx_test_geno]
    train_geno = [int_genotypes[ii] for ii in idx_train_geno]
    
    geno = poly.fit_transform([[2*int(a) - 1 for a in f'{b:0{L}b}'] for b in train_geno])

    result_opt = scipy.linalg.lstsq(
        geno, measured_phenotypes[idx_train_geno])
    res = result_opt[0]
    ## measure the r2 on the test and train set
    train_pred = geno @ res
    # Calculating R-squared
    r2_train = r2_score(measured_phenotypes[idx_train_geno], train_pred)
    
    r2_test = 0
    if test_frac != 0:
        poly = PolynomialFeatures(degree, interaction_only=True)
        test_geno = poly.fit_transform([[2*int(a) - 1 for a in f'{b:0{L}b}'] for b in test_geno])
        test_pred = test_geno @ res
        r2_test = r2_score(measured_phenotypes[idx_test_geno], test_pred)
    
    return res, r2_train, r2_test

def MP_BA_inference(measured_phenotypes, order, L):
    """ Hadamard-reference-free method. Only behave correctly with all genotypes.
    """
    coefficients = hadamard(measured_phenotypes, L)
    # reorder the epistatic coefficients so they are in the same order as the other models
    current_order = [tuple([int(x) for x in f'{a:0{L}b}']) for a in range(2**L)]
    poly_order = [a[-1] for a in sorted([(sum(c), *tuple((-np.array(c))), ii) for ii, c in enumerate(current_order)])]
    coefficients_order = coefficients[poly_order]
    poly = PolynomialFeatures(L, interaction_only=True)
    geno = poly.fit_transform([[2*int(a) - 1 for a in f'{b:0{L}b}'] for b in range(2**L)])
    degree_coefs = poly.powers_.sum(axis=1)
    coefficients_order[degree_coefs > order] = 0
    r2 = r2_score(measured_phenotypes, geno@coefficients_order)
    return coefficients_order, r2

In [ ]:
L = 16
order = [''.join([str(a) for a in f'{b:0{L}b}']) for b in range(2**L)]
df['geno'] = pd.Categorical(df['variant'], categories=order, ordered=True)
df = df.sort_values('geno')
df['geno'] = df['geno'].astype(str)

from sklearn.impute import KNNImputer

# we need to impute some of the values, because RBA doesn't work otherwise 
imputer = KNNImputer(n_neighbors=50)
df_imputed = pd.DataFrame(imputer.fit_transform(df[[f'pos{ii}' for ii in range(1, 17)] + ['h1_mean']]),
                          columns=[f'pos{ii}' for ii in range(1, 17)] + ['h1_mean'])
df_imputed['geno'] = df_imputed.apply(lambda r: ''.join([str(int(r[f'pos{ii}'])) for ii in range(1, 17)]), axis=1)
measured_phenotypes = df_imputed.h1_mean
found_genotypes = df_imputed.geno

In [ ]:
import multiprocessing
from tqdm import tqdm
import pandas as pd

def worker_func(func, ii, L):
    local_results = []
    for deg in range(0, 7):
        print(ii, deg, func.__name__)
        _, r2train, r2test = func(measured_phenotypes, found_genotypes, deg, L, 1/5)
        local_results += [{'r2': r2test, 'type': 'test', 'model': func.__name__[:-10], 'degree': deg}]
        local_results += [{'r2': r2train, 'type': 'train', 'model': func.__name__[:-10], 'degree': deg}]
    return local_results

results = []
args = [(RB_LS_inference, ii, L) for ii in range(5)] + [(RFA_inference, ii, L) for ii in range(5)]
for ag in tqdm(args):
    partial_results = worker_func(*ag)
    results.extend(partial_results)

dfresult = pd.DataFrame(results)
mapmodel = {'BA': 'Least squares (0, 1)', 'RFA': 'Least squares (-1, 1)'}
dfresults.model = dfresults.model.map(mapmodel)
dfresult.to_csv('result_norba.csv')

In [ ]:
results = []

for deg in range(0,7):
    print(deg)
    _, r2 = MP_BA_inference(measured_phenotypes.to_numpy(), deg, L)
    results += [{'r2': r2, 'type': 'na', 'model':'Hadamard (-1, 1)','degree': deg}]
    
for deg in range(0, 7):
    print(deg)
    _, r2 = RBA_inference(measured_phenotypes, found_genotypes, deg, L)
    results += [{'r2': r2, 'type': 'na', 'model':'No average (0, 1)','degree': deg}]
dfresults = dfresults.add(pd.DataFrame(results))
dfresult.to_csv('result_v2.csv')

In [ ]:
dfresults = dfresults.add(pd.DataFrame(results))
dfresult.to_csv('result_v2.csv')

In [ ]:
results = []
## Look at the prediction of a linear model CV as the number of coefficients grow
for deg in range(0,7):
    _, r2train, _ = BA_inference(measured_phenotypes,found_genotypes,  deg, L)
    results += [{'r2': r2train, 'type': 'train', 'model': 'BA', 'degree': deg}]

for deg in range(0,7):
    _, r2train, _ = RFA_inference(measured_phenotypes,found_genotypes,  deg, L)
    results += [{'r2': r2train, 'type': 'train', 'model':'RFA','degree': deg}]
        
for deg in range(0,7):
    _, r2 = RBA_inference(measured_phenotypes,found_genotypes,  deg, L)
    results += [{'r2': r2, 'type': 'na', 'model':'RBA','degree': deg}]
        
dfresult_fulldat = pd.DataFrame(results)
dfresult_fulldat.to_csv('result_fulldat.csv')

In [ ]:

order5_BA, _, _ = RB_LS_inference(measured_phenotypes,found_genotypes,  5, L, test_frac=0)
order5_RFA,_ , _ = RFA_inference(measured_phenotypes,found_genotypes,  5, L, test_frac=0)
order5_RBA, _ = RBA_inference(measured_phenotypes,found_genotypes,  5, L)
poly = PolynomialFeatures(1, interaction_only=True)
geno = poly.fit_transform([[int(a) for a in f'{b:0{L}b}'] for b in range(2**L)])

In [ ]:
np.save('order5_BA.npy', order5_BA)
np.save('order5_RBA.npy', order5_RBA)
np.save('order5_RFA.npy', order5_RFA)
np.save('order5_MP_BA.npy', order5_MP_BA)
np.save('H1_measured_phenotypes.npy', measured_phenotypes)